In [1]:
import sys
import numpy as np

In [2]:
import h5py
import os
from os import path
import datetime

In [3]:
import prince_data_utils.photo_nuclear as ppn
import prince_data_utils.photon_fields as ppf

In [4]:
h5fname = 'prince_db_05.h5'
try:
    prince_db.close()
except:
    pass
output_dir = path.abspath('.')
#exclusively create the file
if path.isfile(path.join(output_dir,h5fname)):
    os.unlink(path.join(output_dir,h5fname))
prince_db = h5py.File(path.join(output_dir,h5fname),'x')
prince_db.attrs['created_on'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
prince_db.attrs['version'] = '0.5.0'
# Make groups for different types of data required by MCEq.
for grp_name in ["photo_nuclear", "EBL_models"]:
    prince_db.create_group(grp_name)
prince_db.keys()

<KeysViewHDF5 ['EBL_models', 'photo_nuclear']>

## Store photo-nuclear interaction tables

In [5]:
talys = ppn.CrossSectionsFromAscii('CRP2_TALYS')
fluka = ppn.CrossSectionsFromAscii('PEANUT_IAS')

CrossSectionsFromAscii::_load(): Loading files D:\OneDrive\devel\git\PriNCe-data-utils\prince_data_utils\resources\photo-nuclear\CRP2_TALYS_*
CrossSectionsFromAscii::_load(): Loading files D:\OneDrive\devel\git\PriNCe-data-utils\prince_data_utils\resources\photo-nuclear\PEANUT_IAS_*


In [6]:
for modname, pnmod in [('CRP2_TALYS', talys), ('PEANUT_IAS', fluka)]:
    hdf_group = prince_db["photo_nuclear"].create_group(modname)
    hdf_group.create_dataset('energy_grid', data=pnmod.energy_grid,
        chunks=True, compression="lzf", shuffle=True)
    hdf_group.create_dataset('inel_mothers', data=pnmod.inel_mothers,
        chunks=True, compression="lzf", shuffle=True)
    hdf_group.create_dataset('mothers_daughters', data=pnmod.mothers_daughters,
        chunks=True, compression="lzf", shuffle=True)
    hdf_group.create_dataset('inelastic_cross_sctions', data=pnmod.inelastic_cross_sctions,
        chunks=True, compression="lzf", shuffle=True)
    hdf_group.create_dataset('fragment_yields', data=pnmod.fragment_yields,
        chunks=True, compression="lzf", shuffle=True)

In [7]:
print(prince_db.keys())
print(prince_db['photo_nuclear'].keys())
print(prince_db['photo_nuclear']['CRP2_TALYS'].keys())

<KeysViewHDF5 ['EBL_models', 'photo_nuclear']>
<KeysViewHDF5 ['CRP2_TALYS', 'PEANUT_IAS']>
<KeysViewHDF5 ['energy_grid', 'fragment_yields', 'inel_mothers', 'inelastic_cross_sctions', 'mothers_daughters']>


## Store EBL photon field arrays (for spline construction)

In [8]:
for ebl_obj in [ppf.Francescini2008(), ppf.Inoue2013(),
                ppf.Gilmore2011(), ppf.Dominguez2010()]:
    ebl_gr = prince_db['EBL_models'].create_group(ebl_obj.__class__.__name__)
    xyz_d = ebl_obj.get_xyz()
    for subset in xyz_d:
        sub_gr = ebl_gr.create_group(subset)
        sub_gr.create_dataset("x", data=xyz_d[subset][0],
        chunks=True, compression="lzf", shuffle=True)
        sub_gr.create_dataset("y", data=xyz_d[subset][1],
        chunks=True, compression="lzf", shuffle=True)
        sub_gr.create_dataset("z", data=xyz_d[subset][2],
        chunks=True, compression="lzf", shuffle=True)
        

In [9]:
print(prince_db['EBL_models'].keys())
print(prince_db['EBL_models']['Dominguez2010'].keys())
print(prince_db['EBL_models']['Dominguez2010']['base'].keys())

<KeysViewHDF5 ['Dominguez2010', 'Francescini2008', 'Gilmore2011', 'Inoue2013']>
<KeysViewHDF5 ['base', 'lower', 'upper']>
<KeysViewHDF5 ['x', 'y', 'z']>


In [10]:
prince_db.close()